<a href="https://colab.research.google.com/github/pareshrchaudhary/experiments/blob/main/fundamentals/MLE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Maximum Likelihood Estimation

**Observation**: We have observed data $(X_1, X_2, \ldots, X_n)$ which are drawn IID (Independently and Identically Distributed) from a probability distribution $f(x; \theta)$, where $\theta$ is a parameter and there exists some “true” value of this parameter denoted as $\Theta=\Theta^*$.

**Likelihood Function**: The Likelihood function $L_n(\theta)$ is defined as the product of all individual probabilities for each observed data point given the parameter $\theta$:

$L_n (\Theta)=\prod_{i=1}^{n} f(X_i;\Theta) $

**Log-Likelihood Function**: The Log-Likelihood function $l_n(\theta)$ is then defined as the natural logarithm of the likelihood function:

$l_n (\Theta)=log(L_n (\Theta))=\sum_{i=1}^{n} log(f(X_i;\Theta)) $

Taking logarithm simplifies calculations and turns products into sums.

**Maximum Likelihood Estimator** $MLE$: The Maximum Likelihood Estimator $(MLE)$ is then calculated by finding the value of theta that maximizes this log-likelihood function:

$\hat{\Theta}_{MLE}=argmax_\Theta l_n (\Theta)$


In [65]:
# @title 1) Observations: Function to simulate the number of customers arriving at a store
import numpy as np
import torch
from torch.autograd import Variable
import numpy.random as nprandom
np.random.seed(0)  # for reproducibility

def simulate_customer_counts(hour_count, avg_customers_per_hour):
    """
    Simulates customer counts using Poisson distribution.
    - P(X = k) = (lambda^k * e^(-lambda)) / k!
    - hour_count: Number of hours to simulate
    - avg_customers_per_hour: Average customers arriving per hour
    - Returns simulated customer counts for each hour
    """
    customer_counts = np.zeros(hour_count, dtype=int)

    # Generate simulated customer counts for each hour
    for hour in range(hour_count):
        rate = avg_customers_per_hour  # Average rate of occurrence
        count = 0  # Initialize the count of occurrences
        probability = 1.0  # Initialize the probability of occurrence
        exp_term = np.exp(-rate)  # Exponential term of the Poisson distribution

        # Generate a Poisson-distributed random variable for each hour
        while probability > exp_term:
            count += 1  # Increment the count of occurrences
            probability *= nprandom.rand()  # Generate a random number between 0 and 1
        customer_counts[hour] = count - 1  # Store the simulated count for the current hour

    return customer_counts

# Generate synthetic data: number of customers arriving at the store for each hour
hour_count = 10     # @param {type:"int"} Simulate for 24 hours (one day)
avg_customers_per_hour_true = 50  # @param {type:"int"} True average number of customers per hour
customer_counts = simulate_customer_counts(hour_count, avg_customers_per_hour_true)
customer_counts_true = Variable(torch.tensor(customer_counts, dtype=torch.float32))
print("customer_counts_true = {}".format(customer_counts_true))

customer_counts_true = tensor([55., 41., 61., 44., 51., 50., 42., 55., 43., 57.])


In [67]:
# @title 2) Log-Likelihood Function
# Log-likelihood function for Poisson distribution
def log_likelihood(observed_customer_counts, avg_customers_per_hour):
  '''
    Poisson distribution formula: P(X = k) = (lambda^k * e^(-lambda)) / k!

    Taking the logarithm:
    log(P(X = k)) = log(lambda^k) + log(e^(-lambda)) - log(k!)

    After applying logarithm properties:
    log(P(X = k)) = k * log(lambda) + (-lambda) - log(k!)

  '''
  #  logarithm of the factorial of observed_customer_counts
  log_factorial_customer_counts = torch.lgamma(observed_customer_counts + 1)

  #  logarithm of the Poisson probability mass function (PMF)
  log_poisson_pmf = observed_customer_counts * torch.log(avg_customers_per_hour) \
                    - avg_customers_per_hour - log_factorial_customer_counts

  # Sum up the log probabilities to get the total log-likelihood
  log_likelihood_value = torch.sum(log_poisson_pmf)

  return -log_likelihood_value

In [75]:
# @title 3) Maximum Likelihood Estimation

# Initial guess for the parameter (average customers per hour)
initial_guess = 30.0 # @param
initial_guess = Variable(torch.tensor([initial_guess]), requires_grad=True)

# Set the learning rate for gradient descent
learning_rate = 0.1  # @param
# Set the number of iterations for gradient descent
num_iterations = 1000  # @param

# Gradient descent optimization using PyTorch
for i in range(num_iterations):
    # Compute the gradient of the log-likelihood function with respect to the parameter
    gradient = torch.autograd.grad(log_likelihood(initial_guess, customer_counts_true), initial_guess)[0]

    # Update the parameter using the gradient descent update rule
    initial_guess.data -= learning_rate * gradient.data

# Extract the estimated parameter
avg_customers_per_hour_estimate = initial_guess.data.item()

print("Estimated average customers per hour:", avg_customers_per_hour_estimate)

Estimated average customers per hour: 48.94633483886719
